# 1. Get DataSet

In [13]:
# data directory
data_root='../sleep_lab_data'
human_name='1_6_years_old_Alessandro_Becker'

In [14]:
# load data
from SleepDataParser import SleepDataParser

osp = SleepDataParser(data_root, human_name)
print(osp.data_root, osp.human_name)
x_data, y_data =  osp.get_dataset()

../sleep_lab_data 1_6_years_old_Alessandro_Becker
(16107, 400)
(16107, 800)
(16107, 1200)
(16107, 1600)
(16107, 2000)
(16107, 2400)
(16107, 2800)
(16107, 3200)
(16107, 3600)
(16107, 4000)
(16107, 4400)
(16107,)


In [15]:
# cut to test & train
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.20, random_state=42)

In [16]:
# check data shape
x_data.shape, x_train.shape

((16107, 4400), (12885, 4400))

In [17]:
# check y (valid if num)
set(y_train)

{0, 1, 2, 3, 4}

# 2. DCNN model

In [18]:
from keras.layers import Input, Deconvolution2D, Conv2DTranspose
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from sklearn.datasets import load_digits
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation
from keras.layers import Convolution2D as Conv2D
from keras.layers import MaxPooling2D
from keras import backend as K

In [19]:
# set basic parameters

batch_size = 128
num_classes = len(set(y_data))
epochs = 20 # training times

# input image dimensions
img_rows, img_cols = 40,int(x_data.shape[1]/40)
img_rows, img_cols

(40, 110)

In [20]:
# data reshape to tensorflow format
if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (12885, 40, 110, 1)
12885 train samples
3222 test samples


In [21]:
# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [25]:
# create DCNN
model = Sequential()

model.add(Deconvolution2D(64, kernel_size=(2,2), activation='relu', input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Deconvolution2D(32, kernel_size=(2,2), activation='relu', input_shape=input_shape))
model.add(Flatten())

model.add(Dense(158, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

In [26]:
# train and get acc
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1)
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Epoch 1/20
12885/12885 [==============================] - 114s 9ms/step - loss: 0.9351 - acc: 0.6995
Epoch 2/20
12885/12885 [==============================] - 115s 9ms/step - loss: 0.7312 - acc: 0.7561
Epoch 3/20
12885/12885 [==============================] - 117s 9ms/step - loss: 0.6790 - acc: 0.7692
Epoch 4/20
12885/12885 [==============================] - 112s 9ms/step - loss: 0.6562 - acc: 0.7752
Epoch 5/20
12885/12885 [==============================] - 112s 9ms/step - loss: 0.6327 - acc: 0.7819
Epoch 6/20
12885/12885 [==============================] - 112s 9ms/step - loss: 0.6149 - acc: 0.7853
Epoch 7/20
12885/12885 [==============================] - 112s 9ms/step - loss: 0.5957 - acc: 0.7928
Epoch 8/20
12885/12885 [==============================] - 111s 9ms/step - loss: 0.5790 - acc: 0.7938
Epoch 9/20
12885/12885 [==============================] - 111s 9ms/step - loss: 0.5609 - acc: 0.7970
Epoch 10/20
12885/12885 [==============================] - 112s 9ms/step - loss: 0.5473 - a

In [24]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_transpose_3 (Conv2DTr (None, 41, 111, 5)        25        
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 20, 55, 5)         0         
_________________________________________________________________
conv2d_transpose_4 (Conv2DTr (None, 21, 56, 3)         63        
_________________________________________________________________
flatten_2 (Flatten)          (None, 3528)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 158)               557582    
_________________________________________________________________
dropout_2 (Dropout)          (None, 158)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 5)                 795       
Total para